In [4]:
import pandas

In [7]:
iecc_county_shapefiles = pandas.read_csv('iecc_county_shapefiles.csv')
iecc_county_degree_days = pandas.read_csv('iecc_county_degree_days.csv')

In [9]:
iecc_county_shapefiles.head()

,County Name,State-County,Unnamed: 2,State Abbr.,geometry,Geographic Name,County,State,IECC Climate Zone,Climate Zone #
0,Autauga,AL-Autauga,Autauga,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,"Autauga County, Alabama",Autauga,Alabama,3A,3
1,Baldwin,AL-Baldwin,Baldwin,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,"Baldwin County, Alabama",Baldwin,Alabama,2A,2
2,Barbour,AL-Barbour,Barbour,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,"Barbour County, Alabama",Barbour,Alabama,3A,3
3,Bibb,AL-Bibb,Bibb,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,"Bibb County, Alabama",Bibb,Alabama,3A,3
4,Blount,AL-Blount,Blount,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,"Blount County, Alabama",Blount,Alabama,3A,3


In [10]:
iecc_county_degree_days.head()

,state,county,hdd65,cdd65,hdd50,cdd50,cdh80,climate_zone,moisture_regime
0,Alabama,Autauga,2229,2208,528,5983,8755,3,moist
1,Alabama,Baldwin,1773,2394,367,6463,7380,2,moist
2,Alabama,Barbour,2368,2031,590,5729,6930,3,moist
3,Alabama,Bibb,2699,1805,745,5326,5735,3,moist
4,Alabama,Blount,3400,1502,1096,4639,5281,3,moist


In [12]:
iecc_county_degree_days = iecc_county_degree_days.merge(iecc_county_shapefiles, how='left', left_on=['state','county'], right_on=['State','County'])

In [16]:
iecc_county_degree_days = iecc_county_degree_days.drop(['state','county','County Name','State-County','Unnamed: 2','Geographic Name'], axis=1)
iecc_county_degree_days.head()

,hdd65,cdd65,hdd50,cdd50,cdh80,climate_zone,moisture_regime,State Abbr.,geometry,County,State,IECC Climate Zone,Climate Zone #
0,2229,2208,528,5983,8755,3,moist,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,Autauga,Alabama,3A,3
1,1773,2394,367,6463,7380,2,moist,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,Baldwin,Alabama,2A,2
2,2368,2031,590,5729,6930,3,moist,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,Barbour,Alabama,3A,3
3,2699,1805,745,5326,5735,3,moist,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,Bibb,Alabama,3A,3
4,3400,1502,1096,4639,5281,3,moist,AL,<Polygon><outerBoundaryIs><LinearRing><coordin...,Blount,Alabama,3A,3


In [17]:
iecc_county_degree_days.to_csv('iecc_county_degree_days_with_shapes.csv')

###Drop counties outside of PNW

In [96]:
df = iecc_county_degree_days[iecc_county_degree_days['State Abbr.'].isin(['WA','OR','ID','MT'])]

In [112]:
df.loc[(df.hdd65 <= 6000),'BPA Heating CZ'] = 1
df.loc[((df.hdd65 >= 6001) & (df.hdd65 <= 7499)),'BPA Heating CZ'] = 2
df.loc[(df.hdd65 >= 7500),'BPA Heating CZ'] = 3

df.loc[(df.cdd65 <= 300),'BPA Cooling CZ'] = 1
df.loc[((df.cdd65 >= 301) & (df.cdd65 <= 599)),'BPA Cooling CZ'] = 2
df.loc[(df.cdd65 >= 600),'BPA Cooling CZ'] = 3

df.loc[((df['BPA Heating CZ'] == 1) & (df['BPA Cooling CZ'] == 1) & (df['State Abbr.'] == 'WA')),'BPA ResStock CZ'] = 'H1C1 WA'
df.loc[((df['BPA Heating CZ'] == 1) & (df['BPA Cooling CZ'] == 1) & (df['State Abbr.'] == 'OR')),'BPA ResStock CZ'] = 'H1C1 OR'
df.loc[((df['BPA Heating CZ'] == 1) & (df['BPA Cooling CZ'] == 2)),'BPA ResStock CZ'] = 'H1C2'
df.loc[((df['BPA Heating CZ'] == 1) & (df['BPA Cooling CZ'] == 3)),'BPA ResStock CZ'] = 'H1C3'
df.loc[((df['BPA Heating CZ'] == 2)),'BPA ResStock CZ'] = 'H2'
df.loc[((df['BPA Heating CZ'] == 3)),'BPA ResStock CZ'] = 'H3'

In [113]:
cats = list(set(df['BPA ResStock CZ']))
cats.sort()
colors = ['#ffffcc','#c7e9b4','#7fcdbb','#41b6c4','#2c7fb8','#253494'] # from colorbrewer2.org
colors = [color + 'CC' for color in colors] # append opacity values (in hex)
colormap = dict(zip(cats, colors))
colormap

{'H1C1 OR': '#ffffccCC',
 'H1C1 WA': '#c7e9b4CC',
 'H1C2': '#7fcdbbCC',
 'H1C3': '#41b6c4CC',
 'H2': '#2c7fb8CC',
 'H3': '#253494CC'}

In [114]:
df.loc[:,'BPA ResStock CZ Color'] = df.loc[:,'BPA ResStock CZ'].replace(colormap)

In [115]:
df.head()

,hdd65,cdd65,hdd50,cdd50,cdh80,climate_zone,moisture_regime,State Abbr.,geometry,County,State,IECC Climate Zone,Climate Zone #,BPA Heating CZ,BPA Cooling CZ,BPA ResStock CZ,BPA ResStock CZ Color
556,5878,726,2497,2768,3410,5,dry,ID,<Polygon><outerBoundaryIs><LinearRing><coordin...,Ada,Idaho,5B,5,1,3,H1C3,#41b6c4CC
557,7412,477,3575,2088,2338,6,dry,ID,<Polygon><outerBoundaryIs><LinearRing><coordin...,Adams,Idaho,6B,6,2,2,H2,#2c7fb8CC
558,7197,418,3462,2136,841,6,dry,ID,<Polygon><outerBoundaryIs><LinearRing><coordin...,Bannock,Idaho,6B,6,2,2,H2,#2c7fb8CC
559,8437,201,4360,1586,159,6,dry,ID,<Polygon><outerBoundaryIs><LinearRing><coordin...,Bear Lake,Idaho,6B,6,3,1,H3,#253494CC
560,6762,367,2874,1940,1381,5,dry,ID,<Polygon><outerBoundaryIs><LinearRing><coordin...,Benewah,Idaho,5B,5,2,2,H2,#2c7fb8CC


In [116]:
df.to_csv('pnw_county_degree_days_with_shapes.csv')